## Machine Learning Project

### Importing the Libraries and Packages

In [104]:
import pandas as pd
import datetime
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from math import ceil
from pandas_profiling import ProfileReport
import scipy.cluster.hierarchy as shc
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'


import warnings
warnings.filterwarnings("ignore")

### Some Functions that we are going to use

In [122]:
def questionmark_to_nan(series):
    '''
    This function turns a binary variable with values ' ' or B in to 0 or 1
    It needs as input the dataframe and the column name 
    (column name needs to be between '')
    '''
    series = series.map(lambda x: np.nan if x == '?' else x)
    return series

def from_object_to_datetime(series):
    series = pd.to_datetime(series, infer_datetime_format=True)
    return series

def longmonth_to_number(df, name):
    conditions = [(train["MOB"]== "January"), (train["MOB"]== "February"), (train["MOB"]== "March"), (train["MOB"]== "April"), 
                 (train["MOB"]== "May"),(train["MOB"]== "June"), (train["MOB"]== "July"), (train["MOB"]== "August"), (train["MOB"]== "September")
                 (train["MOB"]== "October"),(train["MOB"]== "November"),(train["MOB"]== "December")]
    choices = [1,2,3,4,5,6,7,8,9,10,11,12]
    train["MOB"] = np.select(conditions, choices, default=np.nan)
    return train["MOB"]

In [4]:
train = pd.read_excel("Train.xlsx")
test = pd.read_excel("Test.xlsx")

In [124]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22400 entries, 0 to 22399
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CITIZEN_ID              22400 non-null  int64 
 1   Name                    22400 non-null  object
 2   Birthday                22400 non-null  object
 3   Native Continent        22400 non-null  object
 4   Marital Status          22400 non-null  object
 5   Lives with              22400 non-null  object
 6   Base Area               22400 non-null  object
 7   Education Level         22400 non-null  object
 8   Years of Education      22400 non-null  int64 
 9   Employment Sector       22400 non-null  object
 10  Role                    22400 non-null  object
 11  Working Hours per week  22400 non-null  int64 
 12  Money Received          22400 non-null  int64 
 13  Ticket Price            22400 non-null  int64 
 14  Income                  22400 non-null  int64 
 15  MO

In [123]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
CITIZEN_ID,22400.0,23685.500000,6466.467351,12486.0,18085.75,23685.5,29285.25,34885.0
Years of Education,22400.0,13.173884,2.512451,2.0,12.00,13.0,15.00,21.0
Working Hours per week,22400.0,40.483795,12.370921,1.0,40.00,40.0,45.00,99.0
Money Received,22400.0,1324.915357,9227.771813,0.0,0.00,0.0,0.00,122999.0
Ticket Price,22400.0,109.145313,500.208904,0.0,0.00,0.0,0.00,5358.0
Income,22400.0,0.237098,0.425313,0.0,0.00,0.0,0.00,1.0


#### Vamos inspect as variáveis que não são numéricas


In [101]:
#Cleaning of the Data
a = train['Birthday'].str.partition(',')
train["MOB"] = train['Birthday'].str.partition(' ')[0]
train["DOB"] = a[0].str.partition(' ')[2]
train["YOB"] = train['Birthday'].str.partition(',')[2]

In [121]:
train[["DOB","MOB","YOB"]]

,DOB,MOB,YOB
0,1,July,2003
1,25,January,2006
2,10,May,2009
3,25,March,1985
4,29,May,2015
...,...,...,...
22395,15,March,2023
22396,7,March,2025
22397,20,July,2022
22398,11,June,2011


In [9]:
train['Native Continent'].value_counts()

Europe     19111
Africa      2187
Asia         699
America      219
Oceania      184
Name: Native Continent, dtype: int64

In [10]:
train['Marital Status'].value_counts()

Married                         10219
Single                           7416
Divorced                         3072
Separated                         699
Widow                             696
Married - Spouse Missing          284
Married - Spouse in the Army       14
Name: Marital Status, dtype: int64

In [12]:
train['Lives with'].value_counts()

Wife               9012
Other Family       5777
Children           3519
Alone              2362
Husband            1049
Other relatives     681
Name: Lives with, dtype: int64

In [13]:
train['Base Area'].value_counts()

Northbury        20074
Fanfoss            443
?                  395
Alverton           135
Butterpond          96
Watford             83
Auchenshuggle       80
Pran                77
Sharnwick           70
Aroonshire          64
Laewaes             63
Fool's March        59
Eelry               56
Wigston             56
Sharpton            54
Lanercost           52
Cherrytown          51
Aerilon             45
King's Watch        44
Bellmoral           40
Kirkwall            40
Knife's Edge        37
Laenteglos          32
Tranmere            30
Drumchapel          25
Aberuthven          23
Orilon              22
Kald                19
Carlisle            18
MillerVille         17
Willesden           16
Lewes               16
Conriston           12
Marnmouth           11
Middlesbrough       10
Mensfield            9
Woodpine             9
Bellenau             8
Redwick Bush         8
Ironforge            1
Name: Base Area, dtype: int64

In [14]:
train['Education Level'].value_counts()

Professional School                     7232
High School + PostGraduation            4994
Bachelors + PostGraduation              3696
Masters                                 1193
Professional School + PostGraduation     953
High School - 2nd Cycle                  809
Bachelors                                735
High School - 1st Cycle                  649
Middle School - 2nd Cycle                432
Masters + PostGraduation                 397
Middle School Complete                   342
PhD                                      289
High School Complete                     287
Middle School - 1st Cycle                237
Primary School                           122
Preschool                                 33
Name: Education Level, dtype: int64

In [15]:
train['Employment Sector'].value_counts()

Private Sector - Services     15599
Self-Employed (Individual)     1764
Public Sector - Others         1419
?                              1264
Private Sector - Others         880
Self-Employed (Company)         763
Public Sector - Government      692
Unemployed                       12
Never Worked                      7
Name: Employment Sector, dtype: int64

In [16]:
train['Role'].value_counts()

Professor                         2849
Management                        2797
Repair & constructions            2795
Administratives                   2608
Sales                             2531
Other services                    2287
Machine Operators & Inspectors    1384
?                                 1271
Transports                        1071
Cleaners & Handlers                922
Agriculture and Fishing            702
IT                                 626
Security                           450
Household Services                 102
Army                                 5
Name: Role, dtype: int64